<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import scrapbook as sb 
import pandas as pd
import numpy as np 
import seaborn as sns 
import numpy as np
from statistics import mean , median
import matplotlib.pyplot as plt

# Baseline 

In [3]:
books = sb.read_notebooks("./BaseLine_Model_Output")
baseLine_data = []
for nb in books.notebooks: 
    nbList=[nb.scraps['Stats Model MAE'].data,nb.scraps['Catboost MAE'].data]
    baseLine_data.append(nbList)
df = pd.DataFrame(baseLine_data, columns = ["Stats Model","Catboost"])
baseline_data = np.array(baseLine_data)
stats = median(baseline_data[:,0])
catboost = median(baseline_data[:,1])
print(stats)
print(catboost)

0.5634902902217045
0.42156855611876876


# GAN Model

In [4]:
book = sb.read_notebooks("./GAN_Output")
gan_data = []
gan_mse = []
for nb in book.notebooks:
    metrics = nb.scraps['GAN_1 Metrics'].data
    for i in range(1000):
        gan_mse.append(metrics[0][i])
    nbList = [nb.scraps['GAN Model MSE'].data,
              nb.scraps['GAN Model MAE'].data,
              nb.scraps['GAN Model Euclidean distance'].data,
              nb.scraps['GAN Model Manhattan Distance'].data]        
    gan_data.append(nbList)
    
df = pd.DataFrame(gan_data, columns = ['MSE','MAE','Euclidean Distance','Manhattan Distance'])
display(df.style)
print("MEDIAN:")
print(df.median(axis = 0))
gan_data = np.array(gan_data)
gan_median = median(gan_data[:,1])
print(gan_median)

,MSE,MAE,Euclidean Distance,Manhattan Distance
0,3.244939,1.301206,8.030751,26.024117
1,2.417923,1.175627,6.943800,23.512542
2,0.513711,0.622917,3.189449,12.458342
3,2.543862,1.322338,7.105975,26.446752
4,0.546357,0.570272,3.291769,11.405436
5,3.114395,1.325301,7.846411,26.506024
6,1.289169,0.899219,5.058569,17.984390
7,1.141973,0.777867,4.742538,15.557348
8,6.274438,1.835719,11.176526,36.714379
9,4.937372,1.792841,9.924780,35.856830


MEDIAN:
MSE                    2.480892
MAE                    1.238416
Euclidean Distance     7.024887
Manhattan Distance    24.768330
dtype: float64
1.2384164835370206


# ABC_GAN Analysis

## ABC Pre-generator - Catboost 


In [5]:
book = sb.read_notebooks("./ABC_GAN_Catboost")
paramVal = [[1,1],[1,0.1],[1,0.01],[1,0],[0.1,1],[0.1,0.1],[0.1,0.01],[0.1,0],[0.01,1],[0.01,0.1],[0.01,0.01],[0.01,0]]
abc_mae = [[] for i in range(12)]
abc_mae_skip = [[] for i in range(12)]
abc_mae_mean = [[] for i in range(12)]
abc_mae_skip_mean = [[] for i in range(12)]
abc_weights = [[] for i in range(12)]
prior_model = [[] for i in range(12)]
abc_pre_generator = [[] for i in range(12)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[1]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[0]['value'])
    # #Divide data according to parameters 
    for i in range(12):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))   

In [6]:

data = [[] for i in range(12)]
for i in range(12):
    for j in range(len(abc_weights[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_mae_mean[i][j]])
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE','ABC pre-generator MAE','ABC GAN MAE'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,ABC GAN MAE
0,1,1,0.26219,1.22593,0.34578
1,1,1,0.27885,2.60982,0.56436
2,1,1,0.30797,2.38343,0.44850


Variance                 1.000000
Bias                     1.000000
Prior Model MAE          0.278853
ABC pre-generator MAE    2.383430
ABC GAN MAE              0.448500
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,ABC GAN MAE
0,1,0.1,0.18589,0.84451,0.38985
1,1,0.1,1.32055,2.79608,0.70250
2,1,0.1,0.33134,1.66406,0.42781


Variance                 1.000000
Bias                     0.100000
Prior Model MAE          0.331341
ABC pre-generator MAE    1.664063
ABC GAN MAE              0.427805
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,ABC GAN MAE
0,1,0.01,0.41742,1.66583,0.53523
1,1,0.01,0.18277,1.16504,0.44517
2,1,0.01,0.08807,0.55664,0.36035


...

Variance                 0.010000
Bias                     0.100000
Prior Model MAE          0.333162
ABC pre-generator MAE    0.321187
ABC GAN MAE              0.431642
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,ABC GAN MAE
0,0.01,0.01,0.35389,0.35468,0.47046
1,0.01,0.01,0.45515,0.46258,0.43940
2,0.01,0.01,0.16421,0.16281,0.31283


Variance                 0.010000
Bias                     0.010000
Prior Model MAE          0.353886
ABC pre-generator MAE    0.354683
ABC GAN MAE              0.439399
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,ABC GAN MAE
0,0.01,0,0.07043,0.07119,0.24953
1,0.01,0,0.35054,0.34863,0.44722
2,0.01,0,0.23040,0.23036,0.31068


Variance                 0.010000
Bias                     0.000000
Prior Model MAE          0.230401
ABC pre-generator MAE    0.230359
ABC GAN MAE              0.310684
dtype: float64
-------------------------------------------------------------


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [7]:
# Display TabNet Summary Tables 
data = np.array(data)
catboostData = []
for i in range(12): 
    data[i] = np.array(data[i])
    catboostData.append([paramVal[i][0], paramVal[i][1],catboost,median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
df = pd.DataFrame(catboostData, columns = ['Variance','Bias','TabNet','Prior Model MAE','mGAN','skipGAN','Skip Node weight'])
display(df.round(5))

IndexError: index 5 is out of bounds for axis 1 with size 5

## ABC Pre-generator - Stats 


In [ ]:
book = sb.read_notebooks("./ABC_GAN_Stats")
paramVal = [[1,1],[1,0.1],[1,0.01],[1,0],[0.1,1],[0.1,0.1],[0.1,0.01],[0.1,0],[0.01,1],[0.01,0.1],[0.01,0.01],[0.01,0]]
abc_mae = [[] for i in range(12)]
abc_mae_skip = [[] for i in range(12)]
abc_mae_mean = [[] for i in range(12)]
abc_mae_skip_mean = [[] for i in range(12)]
abc_weights = [[] for i in range(12)]
prior_model = [[] for i in range(12)]
abc_pre_generator = [[] for i in range(12)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[1]['value'])
    #Divide data according to parameters 
    for i in range(9):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [ ]:
data = [[] for i in range(12)]
for i in range(12):
    for j in range(len(abc_pre_generator[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_mae_mean[i][j]])
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE','ABC pre-generator MAE','ABC GAN MAE'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

In [ ]:
# Display Stats Summary Tables 
data = np.array(data)
catboostData = []
for i in range(9): 
    catboostData.append([gan_median,stats,paramVal[i][0], paramVal[i][1],median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
df = pd.DataFrame(catboostData, columns = ['GAN','Stats','Variance','Bias','Prior Model','mGAN','skipGAN','Weight'])
display(df.round(5))